In [ ]:

import sys
sys.path.append("././quaos")
from math import gcd

from paulis import PauliSum, PauliString, Pauli
import numpy as np
from gates import Circuit, SUM as CX, Hadamard as H, PHASE as S, SWAP
from circuits.utils import solve_modular_linear
from circuits.known_circuits import to_x, to_ix
from circuit_utils import symplectic_effect, random_clifford
from hamiltonian import random_pauli_hamiltonian, cancel_pauli, symplectic_pauli_reduction, pauli_reduce

Identify $V$ in $H = H_0 + V$ such that $\exists$ a Clifford circuit $C$ where $II \cdots IZ$ is a symmetry of $CH_0C^\dagger$

$V = \sum_i c_i Q_i$ with $Q_i = ***\cdots **X$ are all Paulis in $CHC^\dagger$ with a leading $X$.

We want to find $C$.

In [ ]:
   
def transform_to_basis(pauli_sum: PauliSum, indexes_already_in_basis: list[int]) -> tuple[Circuit, PauliSum]:
    circuit = Circuit(dims=pauli_sum.dimensions)
    basis_elements_max = 2 * pauli_sum.n_qudits() - len(indexes_already_in_basis)
    if basis_elements_max > pauli_sum.n_paulis():
        basis_elements_max = pauli_sum.n_paulis()
    
    for pauli_index in range(basis_elements_max):
        for qudit_index in range(pauli_sum.n_qudits()):
            if pauli_index not in indexes_already_in_basis:
                circuit = to_x(pauli_sum[pauli_index], pauli_index, indexes_already_in_basis)
                pauli_sum = circuit.act(pauli_sum)
    return circuit, pauli_sum


def find_leading_x(pauli_sum: PauliSum) -> tuple[list[int], int]:
    leading_xs = []
    if np.any(pauli_sum.x_exp):
        for i, p in enumerate(pauli_sum.pauli_strings):
            xs = p.x_exp
            if np.any(xs):
                leading_x = len(xs) - 1 - np.argmax(xs[::-1] != 0)  # furthest x to right hand side
            else:
                leading_x = None
            leading_xs.append(leading_x)
        lx_tuple = [(i, v) for i, v in enumerate(leading_xs) if v is not None]
        max_val = max(v for _, v in lx_tuple)
        indices = [i for i, v in lx_tuple if v == max_val]
        return indices, int(max_val)
    else:
        print('no leading x')
        return [], -1


def lowest_z_only(pauli_sum: PauliSum) -> int:
    z_pauli = 0
    for i in range(pauli_sum.n_paulis()):
        min_zs = pauli_sum.n_qudits()
        if pauli_sum[i].is_z():
            if sum(pauli_sum[i].z_exp) < min_zs:
                min_zs = sum(pauli_sum[i].z_exp)
                z_pauli = i
    return z_pauli

def lowest_x_only(pauli_sum: PauliSum) -> int:
    x_pauli = 0
    for i in range(pauli_sum.n_paulis()):
        min_xs = pauli_sum.n_qudits()
        if pauli_sum[i].is_x():
            if sum(pauli_sum[i].x_exp) < min_xs:
                min_xs = sum(pauli_sum[i].x_exp)
                x_pauli = i
    return x_pauli


def inv_fourier_sum_fourier(dimensions: list[int], fourier_index: int, control: int, target: int) -> Circuit:
    """
        If fourier_index=0 control=0, target=1 this is a CZ
        This has the nice effect of:

        x0zs xrz0 -> x0z{r+s} xrz0
        xr'z0 ... -> xr'z0 ...

        If fourier_index=1 control=0, target=1
        This has the nice effect of:
        
    """
    c = Circuit(dimensions)
    c.add_gate(H(fourier_index, dimensions[fourier_index], inverse=True))
    c.add_gate(CX(control, target, dimensions[control]))
    c.add_gate(H(fourier_index, dimensions[fourier_index]))
    return c

def random_pauli_string(dimensions: list[int]) -> PauliString:
    """Generates a random PauliString of n_qudits"""
    n_qudits = len(dimensions)
    x_array = np.zeros(n_qudits, dtype=int)
    z_array = np.zeros(n_qudits, dtype=int)
    for i in range(n_qudits):
        x_array[i] = np.random.randint(0, dimensions[i])
        z_array[i] = np.random.randint(0, dimensions[i])
    p_string = PauliString(x_array, z_array, dimensions=dimensions)

    return p_string


In [ ]:
def remake_basis_select_xz(pauli_sum: PauliSum, x_pauli: int, z_pauli: int, target_qudit: int):
    """
    The goal of this function is to move the pauli_sum to the form (target qudit moved to position 0)
    index    |          pauli         
    x_pauli  | xrz0 x0z0 x0z0 ... x0z0
    z_pauli  | x0zs x0z0 x0z0 ... x0z0
      ...    | x0z0 xrz0 x0z0 ... x0z0
      ...    | x0z0 x0zs x0z0 ... x0z0
      ...    | x0z0 x0z0 xrz0 ... x0z0
    
    
    Until we have a full basis. There are often some skipped rows, due to commutation rules.

    If x_pauli and z_pauli do not anti-commute then we must add a contribution to another pauli (chosen to be 1) that 
    alters the commutation, allowing the desired form of qudit 0.

    """
    
    c = to_ix(pauli_sum[x_pauli], target_qudit)
    # x_pauli now xrz0 x0z0 x0z0 ... x0z0
    if target_qudit != 0:
        c.add_gate(SWAP(target_qudit, 0, pauli_sum.dimensions[target_qudit]))
    pauli_sum = c.act(pauli_sum)

    # now we check the commutation of the two specified paulis - if they anticommute we look for desired form
    # start by making the z_pauli * X I I I I    - useful in either case
    c_temp = to_ix(pauli_sum[z_pauli], 1, ignore=[0])
    pauli_sum = c_temp.act(pauli_sum)
    c += c_temp

    commute = pauli_sum[x_pauli].commute(pauli_sum[z_pauli])
    if commute:
        # we instead make the form 
        # X X I I ...
        # Z Z I I ...
        # We then need a complementary set of paulis to make the basis
        # X Z I I  <-- this one should have the minimal possible coefficient - in principle this still doesn't guarantee
        # Z X I I  |   the minimal weight x components if those outside the basis are dependent on this X 
        #          |   we could catch this case later by going through all paulis to make the X Z out of. 

        raise NotImplementedError('Commutation case not implemented yet')
    
    else:
        # we have anti-commutation
        # first check for Ys on the z_pauli on qudit 0 - remove x components with cnots
        if pauli_sum[z_pauli, 0].z_exp != 0 and pauli_sum[z_pauli, 0].x_exp != 0:
            # z_pauli is YX...I
            print('we have a Y')
            n_cnots = solve_modular_linear(pauli_sum[z_pauli, 0].x_exp, pauli_sum[z_pauli, 1].x_exp,
                                                  pauli_sum.dimensions[0])
            c_temp = Circuit(dimensions=pauli_sum.dimensions)
            for i in range(n_cnots):
                c_temp.add_gate(CX(0, 1, pauli_sum.dimensions[0]))
            pauli_sum = c_temp.act(pauli_sum)
            c += c_temp
            # now should be ZXI...I
        print('now should be ZXI...I')
        print(pauli_sum)

        # we can make the desired form in preamble
        if pauli_sum[z_pauli, 0].z_exp != 0 and pauli_sum[z_pauli, 0].x_exp == 0:
            # z_pauli is ZXI...I 
            n_fourier_sums = solve_modular_linear(pauli_sum[z_pauli, 0].z_exp, pauli_sum[z_pauli, 1].x_exp,
                                                  pauli_sum.dimensions[0])
            c_temp = Circuit(dimensions=pauli_sum.dimensions)
            for i in range(n_fourier_sums):
                c_temp += inv_fourier_sum_fourier(pauli_sum.dimensions, 0, 0, 1)
            pauli_sum = c_temp.act(pauli_sum)
            c += c_temp
            
        else:
            raise Exception("Weird...")
        
    # now we need to clean up the components on the zeroth qudit on the remaining pauli strings that may have been added 
    # by the cnots

    # we make a list of these paulis of the form (index, x_exp, z_exp)
    # list_of_paulis = []
    # for i in range(pauli_sum.n_paulis()):
    #     if i != x_pauli and i != z_pauli:
    #         if pauli_sum[i, 0].x_exp != 0 or pauli_sum[i, 0].z_exp != 0:
    #             list_of_paulis.append((i, pauli_sum[i, 0].x_exp, pauli_sum[i, 0].z_exp))
    
    # # we now loop through the list of paulis and remove the x and z components using cnots and S gates
    # for i, x_exp, z_exp in list_of_paulis:
    #     if x_exp != 0:
    #         # look for an x component in pauli_i and use it to cancel the 0th qudit x
    #         # if there aren't and hadamard a z. 
    #         # (If there are neither we can combine this pauli with the x_pauli or z_pauli, or it is outside the basis)
            

    #     if z_exp == 0:
    #         # same as above but with z
        

    return c
    


In [ ]:
ham = random_pauli_hamiltonian(8, [2] * 6, mode='random')
h_red, conditioned_hamiltonians, C, all_phases = pauli_reduce(ham)


ps = ['x1z0 x0z0', 'x1z1 x2z0']
ps = PauliSum(ps, dimensions=[3, 3])
print(ps)
c = inv_fourier_sum_fourier([3, 3], 1, 1, 0)
cx = Circuit(dimensions=[3, 3])
cx.add_gate(CX(1, 0, 3))
c2 = inv_fourier_sum_fourier([3, 3], 0, 0, 1)

symplectic_effect(c2)


In [ ]:
# c = remake_basis_select_x(conditioned_hamiltonians[0], leading_x_strings[0], leading_x_pauli)
# print(c.act(conditioned_hamiltonians[0]))

print(conditioned_hamiltonians[0])
c = remake_basis_select_xz(conditioned_hamiltonians[0], 0, 1, 2)
print(c.act(conditioned_hamiltonians[0]))

In [ ]:

M_cnot = np.array([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
], dtype=int)

v1 = np.array([1, 0, 0, 0])
v2 = np.array([1, 1, 1, 1])
v3 = np.array([0, 0, 0, 1])
v4 = np.array([0, 1, 1, 0])
v5 = np.array([1, 0, 0, 1])
print((v1.T @ M_cnot @ v1 // 2) % 2)
print((v2.T @ M_cnot @ v2 // 2) % 2)
print((v3.T @ M_cnot @ v3 // 2) % 2)
print((v4.T @ M_cnot @ v4 // 2) % 2)
print((v5.T @ M_cnot @ v5 // 2) % 2)
